<a href="https://colab.research.google.com/github/tmckim/materials-sp24-colab/blob/main/lec_demos/lec10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Before you start - Save this notebook!

When you open a new Colab notebook from the WebCampus (like you hopefully did for this one), you cannot save changes. So it's  best to store the Colab notebook in your personal drive `"File > Save a copy in drive..."` **before** you do anything else.

The file will open in a new tab in your web browser, and it is automatically named something like: "**Copy of lec10.ipynb**". You can rename this to just the title of the assignment "**lec10.ipynb**". Make sure you do keep an informative name (like the name of the assignment) so that you know which files to submit back to WebCampus for grading! More instructions on this are at the end of the notebook.


**Where does the notebook get saved in Google Drive?**

By default, the notebook will be copied to a folder called “Colab Notebooks” at the root (home directory) of your Google Drive. If you use this for other courses or personal code notebooks, I recommend creating a folder for this course and then moving the assignments AFTER you have completed them. <br>

I also recommend you give the folder where you save your notebooks^ a different name than the folder we create below that will store the notebook resources you need each time you work through a course notebook. This includes any data files you will need, links to the images that appear in the notebook, and the files associated with the autograder for answer checking.<br>
You should select a name other than '**NS499-DataSci-course-materials**'. <br>
This folder gets overwritten with each assignment you work on in the course, so you should **NOT** store your notebooks in this folder that we use for course materials! <br><br>For example, you could create a folder called 'NS499-**notebooks**' or something along those lines.
___

### We will now do the setup steps as separate cells to help with issues finding files in google drive/colab. <br> If you restart colab, you must rerun all **5** steps in each of these cells!

In [ ]:
# Step 1
# Setup and add files needed to access gdrive
from google.colab import drive                                   # these lines mount your gdrive to access the files we import below
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# Step 2
# Change directory to the correct location in gdrive (modified way to do this from before)
import os
os.chdir('/content/gdrive/MyDrive/NS499-DataSci-course-materials/')

In [ ]:
# Step 3
# Remove the files that were previously there- we will replace with all the old + new ones for this assignment
!rm -r materials-sp24-colab

In [ ]:
# Step 4
# These lines clone (copy) all the files you will need from where I store the code+data for the course (github)
# Second part of the code copies the files to this location and folder in your own gdrive
!git clone https://github.com/tmckim/materials-sp24-colab '/content/gdrive/My Drive/NS499-DataSci-course-materials/materials-sp24-colab/'

In [ ]:
# Step 5
# Change directory into the folder where the resources for this assignment are stored in gdrive (modified way from before)
os.chdir('/content/gdrive/MyDrive/NS499-DataSci-course-materials/materials-sp24-colab/lec_demos/')

In [ ]:
# Import packages and other things needed
# Don't change this cell; Just run this cell
# If you restart colab, make sure to run this cell again after the first ones above^

from datascience import *
import numpy as np
import warnings
warnings.simplefilter(action='ignore',category=np.VisibleDeprecationWarning)

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")
plt.rcParams["patch.force_edgecolor"] = True

## Learning Objectives ##

Topics:
- Pivots - a special kind of grouping
- Join - combine two tables
- Review apply and lists

## Grouping Review ##

In [ ]:
# Load in our ice cream data
cones = Table.read_table('cones_rating.csv')
cones

Writing the code with `group`

In [ ]:
cones.group('Flavor', sum)

Multi-step based on what you learned prior to `group` in the course.

In [ ]:
# Step 1: Filter dataset to only the chocolate column prices
cones.where('Flavor', are.equal_to('chocolate')).column('Price')

In [ ]:
# Step 2: sum the prices
sum(cones.where('Flavor', are.equal_to('chocolate')).column('Price'))

Using `group` is much more efficient than repeating these steps one by one. However, it is good to know that this is the idea behind what is happening/how `group` is working

We can also `group` by multiple columns. This gives us all unique combinations.

In [ ]:
cones.group(['Flavor', 'Color'])

## Pivot Tables ##

When grouping by two columns it will sometimes be easier to re-arrange the table so that the columns correspond to the values of one of the original grouped columns.

Here is an example:

In [ ]:
# Pivot using the default collect = count
cones.pivot('Color', 'Flavor')

Explicitly naming the function arguments. You wouldn't do this in practice, but it helps to highlight the meaning of the four things we passed to the function above.

In [ ]:
cones.pivot(
    columns = 'Color', # column whose unique values become columns
    rows = 'Flavor', # column whose unique values become rows
    values = 'Price', # column that will be used to do computing
    collect = np.average) # how to aggregate the values

Notice that pivot tables can contain zeros (0). This is because it will tell you that this combination doesn't exist- it shows you all possible combinations. <br>
Group only shows you what exists, so there will usually not be any zeros (0) in the table.

In [ ]:
cones_price = cones.select('Color','Flavor','Price')
cones_price.group(['Color', 'Flavor'], collect=np.average)


## Joins ##

We use join to **join** two tables based on a similar column.

Remember from last time: you can make a table using `.with_rows` instead of `.with_columns`

In [ ]:
# Empty table with columns and no rows
drinks = Table(['Drink', 'Cafe', 'Price'])
drinks

In [ ]:
# drinks table
drinks = drinks.with_rows([
    ['Milk Tea', 'RareTea', 5.5],
    ['Espresso', 'Buzzed Coffee',  1.75],
    ['Latte',    'Buzzed Coffee',  3.25],
    ['Espresso', "Starbucks",   2]
])
drinks

In [ ]:
# discounts table
discounts = Table().with_columns(
    'Coupon % off', make_array(10, 25, 5),
    'Location', make_array('RareTea', 'Buzzed Coffee', 'RareTea')
)
discounts

**Exercise**: Combine the `drinks` and `discounts` tables into a new table which has all possible discounts for each drink.

In [ ]:
combined = drinks.join('Cafe', discounts, 'Location')
combined

What happened to Starbucks?

**Exercise:** Does the order matter?

In [ ]:
# Fill this in
combo =

**Exercise**: Calculate the lowest rate you can get for each drink using the available coupons.

In [ ]:
discounted_frac = 1 - combined.column('Coupon % off') / 100
(
combined.with_column('Discounted Price', (combined.column('Price') * discounted_frac))
.select('Drink', 'Discounted Price')
.group('Drink', min)
)

# Extra group review questions

In [ ]:
nba = Table.read_table('nba_salaries.csv')
nba

Just pick the 2015 season to work with.

In [ ]:
nba_15 = nba.where('season', 2015)
nba_15

Answer the following questions, and use `group` to find the answer.

**Question:** How much money did each team pay for its players' salaries?

**Question:** How many NBA players were there in each of the five positions?

**Question:** What was the average salary of the players at each of the five positions?

### Saving
Remember to save your notebook before closing.
Choose **Save** (and make sure you've already saved a copy in your drive) from the **File** menu.